#### Settings

In [ ]:
target_loc = 'loc_73C16'
formatting = 'SLOPE'

#### Run script

In [ ]:
import re

byte_pattern = re.compile('\\$[0-9A-F]{2}')

In [ ]:
# Extract all byte literals in target_loc

loc_found = False
byte_list = []

with open('../smgp.asm') as f:
    for line in f.readlines():
        if line.startswith(target_loc):
            loc_found = True
        elif loc_found:
            if line.startswith('loc'): # end of target_loc
                break
            byte_list += byte_pattern.findall(line)

In [ ]:
def calculate_curve_chunk_size(offset):
    bytes_left = len(byte_list) - offset
    if bytes_left <= 3:
        return bytes_left
    if byte_list[offset+2] == '$00':
        return 3
    return 5

def calculate_slope_chunk_size(offset):
    bytes_left = len(byte_list) - offset
    if bytes_left <= 3:
        return bytes_left
    if byte_list[offset+2] == '$00':
        return 3
    return 4

calculate_chunk_size = {
    'CURVE': calculate_curve_chunk_size,
    'SLOPE': calculate_slope_chunk_size
}

In [ ]:
# Format in chunks as assembly code reads it

offset = 0

if formatting == 'SLOPE':
    print('\tdc.b\t' + byte_list[0])
    offset = 1

while offset < len(byte_list):
    bytes_left = len(byte_list)-offset
    chunk_size = calculate_chunk_size[formatting](offset)
    print('\tdc.b\t' + ', '.join(byte_list[offset:offset+chunk_size]))
    offset += chunk_size